In [1]:
import requests
# ───── Google Slides 類似検索 API ─────────────────────
from googleapiclient.discovery import build          # type: ignore
from google.oauth2.credentials import Credentials    # type: ignore
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

from langchain_community.vectorstores import Chroma     # type: ignore
from langchain.schema import Document                   # type: ignore
from langchain_openai import OpenAIEmbeddings            # type: ignore

from chromadb.config import Settings
import tempfile, os

PRESENTATION_ID = "1xW8Lze5bfwUzNd9ZqputgTFyQJdoKK3f3I7esGACAds"

def search():
    query = (request.get_json() or {}).get("query", "").strip()
    if not query:
        return jsonify({"error": "query_required"}), 400

    # 認証トークン
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES
            )
            creds = flow.run_local_server(port=0)
        with open("token.json", "w") as f:
            f.write(creds.to_json())

    service = build("slides", "v1", credentials=creds)
    presentation = service.presentations().get(
        presentationId=PRESENTATION_ID
    ).execute()

    # テキスト抽出
    slide_texts: list[str] = []
    for slide in presentation.get("slides", []):
        parts = []
        for elem in slide.get("pageElements", []):
            txt = (
                elem.get("shape", {})
                    .get("text", {})
                    .get("textElements", [])
            )
            for t in txt:
                run = t.get("textRun")
                if run:
                    parts.append(run.get("content", ""))
        slide_texts.append("".join(parts))

    # ベクトル検索
    docs = [
        Document(page_content=slide_texts[i], metadata={"idx": i})
        for i in range(len(slide_texts))
    ]

    store = Chroma.from_documents(
        docs,
        OpenAIEmbeddings(api_key=OPENAI_API_KEY),
        collection_name="slides",
        client_settings=Settings(
            chroma_db_impl="duckdb+parquet",
            persist_directory=TMP_DIR,
        ),
    )
    results = store.similarity_search_with_score(query, k=3)

    return jsonify([
        {
            "slide_index": doc.metadata["idx"] + 1,
            "content": doc.page_content,
            "score": score
        }
        for doc, score in results
    ])


In [2]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv("../.env"))

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
SECRET_KEY     = os.getenv("FLASK_SECRET_KEY", "PLEASE_CHANGE_ME")

SCOPES = ["https://www.googleapis.com/auth/presentations.readonly"]
PRESENTATION_ID = "1xW8Lze5bfwUzNd9ZqputgTFyQJdoKK3f3I7esGACAds"

In [3]:
token_path = "../app/token.json"  # ノートブックと同じフォルダに保存

creds = None
if os.path.exists(token_path):
    creds = Credentials.from_authorized_user_file(token_path, SCOPES)

if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
        creds = flow.run_local_server(port=0)
    with open(token_path, "w") as f:
        f.write(creds.to_json())


In [14]:
service = build("slides", "v1", credentials=creds)
presentation = service.presentations().get(presentationId=PRESENTATION_ID).execute()

slide_texts = []
for slide in presentation.get("slides", []):
    parts = []
    for elem in slide.get("pageElements", []):
        txt = elem.get("shape", {}).get("text", {}).get("textElements", [])
        for t in txt:
            run = t.get("textRun")
            if run:
                parts.append(run.get("content", ""))
    slide_texts.append("".join(parts))


In [15]:
docs = [Document(page_content=text, metadata={"idx": i}) for i, text in enumerate(slide_texts)]

embedding = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
client = PersistentClient(path=TMP_DIR)

store = Chroma(
    client=client,
    collection_name="slides",
    embedding_function=embedding
)

store.add_documents(docs)


NameError: name 'PersistentClient' is not defined

In [ ]:
query = "検索したい内容をここに入力"
results = store.similarity_search_with_score(query, k=3)

for doc, score in results:
    print(f"スライド {doc.metadata['idx'] + 1} | 類似度: {score:.4f}\n{doc.page_content}\n---")


NameError: name 'store' is not defined

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma

llm = ChatOpenAI(temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=store.as_retriever()) # googleスライドの情報を使って質問に答える。

query = "売り上げを上げるには？"
result = qa_chain.run(query)

print("googleスライド内の知識を使って返信を返す。\n", result)




/tmp/ipykernel_78394/3876957001.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0)


NameError: name 'store' is not defined

In [38]:
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)
# OpenAI TTSで音声に変換
response = client.audio.speech.create(
  model = "tts-1",
  voice = "alloy",
  input = result,
)

NameError: name 'result' is not defined

In [44]:
with open("output.mp3", "wb") as f:
    f.write(response.content)

os.system("afplay output.mp3")  # Windowsなら「start」

sh: 1: afplay: not found


32512

# 全体の要約文の取得　

In [13]:
full_presentation_text = "\n\n".join(slide_texts)
full_presentation_text

'販促事例研究\n会員データ軸\n\n\n店頭接客\n配置\n具体\u3000抽象度高く設定することでさらに深ぼっていく\n実際の店舗で起こっているかどうかをずらすことがないように\nVMD\nアプリ\n動線\u3000棚\u3000雑貨店\u3000カゴ\u3000カウンセリング\n\n\n集客\nトライアル\nリピート\n単価(点数)\n機会創出(CEP)\nーーーー施策の概要ーーーー\nーーーー得られる示唆ーーーー\n\n\n集客\nトライアル\nリピート\n単価(点数)\n機会創出(CEP)\nーーーー施策の概要ーーーー\nーーーー得られる示唆ーーーー\n\n\n参考元\nインターンMTG（&mall調査）\n\n\n\n目的変数の分類（）\n\u3000\nビジネス\n直接的\nビジネス間接的\n単一\n（大小）\n組み合わせ\n（割り算）\n売上\n・客数\n・リピート\n・客単価\n（トライアル数／リピート数）\n売上系\n・坪単価\n・\n\n\n＞行動系\n・訪問数\n・滞在時間\n・いいね数\n・投稿数\n・クチコミ数\n・売り場回遊数\n＞感情系\n・満足度\n・次回購買意向\n・推奨意向度\x0b・\n＞行動系\n・立ち寄り率\n・買い上げ率\n・リピート率\n・\x0b・\n＞感情系\n・満足率\n・\x0b・\n＞認知系\n・認知数\n・想起数\n＞認知系\n・認知率\n※分布\n\n\n店舗の売上の構成要素（空間軸）\n24\n店舗の売上高(円)\n①入館者数\n②前面通行量\n③認知率\n④認識率\n⑤入店率\n⑥滞在時間\n⑦買上げ率\n⑧\n⑨一品単価\n⑩\n客単価（円/人）\n「店舗の売上を向上させる」とは、売上の構成要素を上げること。\n店 舗 の 売 上 の 構 成 要 素\nレジ客数（人）\n\n\n\n①やったこと\u3000＋\u3000狙い\u3000＋\u3000結果\n②やったこと\u3000＋\u3000狙い\n\n\n\n\nMEMO\n・集客、トライアル、リピート、単価の4つに事例を振り分け\n\n・店舗とECの2軸だけじゃなく新たに4象限で考える\n\n\n\nMEMO（各ファネルとつながる消費者行動の傾向）\n・什器、店舗レイアウト（デザイン）、店頭での体験→ピークエンドの法則\t\n・POP→フレー

In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(temperature=0.3, api_key=OPENAI_API_KEY)  # 任意のモデルでOK

prompt = PromptTemplate(
    input_variables=["text"],
    template="""
以下はあるプレゼンテーションのスライド全文です。内容をわかりやすく3〜5文程度で要約してください。

スライド本文:
{text}

要約:
"""
)

chain = LLMChain(llm=llm, prompt=prompt)
summary = chain.run(full_presentation_text)

print(summary)


BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 33109 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [15]:
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

# TTS
response = client.audio.speech.create(
  model="tts-1",  # または gpt-4o が対応しているTTSモデル
  voice="alloy",  # 他にも「nova」「shimmer」「echo」などあり
  input=summary,
)

# 保存して再生
with open("summary_output.mp3", "wb") as f:
    f.write(response.content)

os.system("afplay summary_output.mp3")  # Windowsなら os.system("start summary_output.mp3")


NameError: name 'summary' is not defined

In [18]:
from langchain.chains.summarize import load_summarize_chain

# --- LangChainによる階層的要約 ---
docs = [Document(page_content=text) for text in slide_texts]

llm = ChatOpenAI(
    temperature=0.3,
    model_name="gpt-4",  # または gpt-4o / gpt-3.5-turbo-16k
    openai_api_key=OPENAI_API_KEY
)

chain = load_summarize_chain(llm, chain_type="map_reduce")
summary = chain.run(docs)

print("📝 要約結果:\n", summary)

# --- OpenAI TTS による音声出力 ---
response = client.audio.speech.create(
    model="tts-1",         # or "tts-1-hd"
    voice="alloy",         # 選択肢: alloy, nova, shimmer, echo, fable, onyx
    input=summary
)

# 保存して再生
with open("presentation_summary.mp3", "wb") as f:
    f.write(response.content)

# 再生 (Mac/Linux用。Windowsでは"start"コマンドに変更)
os.system("afplay presentation_summary.mp3")

📝 要約結果:
 The article discusses various retail business strategies, including promotional activities, in-store customer service, layout arrangement, and data utilization. It highlights successful marketing strategies used by businesses in Japan, such as visually appealing Point of Purchase (POP) advertising, live commerce, and personalized shopping experiences. The article also emphasizes the importance of understanding consumer behavior and the value customers place on products and services. It mentions the role of AI technology in analyzing customer behavior in physical stores, leading to more effective sales promotion measures.


sh: 1: afplay: not found


32512

In [19]:
# --- ドキュメント変換 ---
docs = [Document(page_content=text) for text in slide_texts]

# --- LLM初期化（gpt-4またはgpt-4o） ---
llm = ChatOpenAI(
    temperature=0.3,
    model_name="gpt-4",
    openai_api_key=OPENAI_API_KEY
)

# --- カスタムプロンプト定義（日本語＋ナレーション調） ---
map_prompt = PromptTemplate.from_template("""
以下のスライドの内容を日本語でナレーション風に要約してください。
聞き手に語りかけるようなトーンで、わかりやすく3〜6文でまとめてください。

スライド内容:
{text}

ナレーション:
""")

combine_prompt = PromptTemplate.from_template("""
以下は複数のスライドのナレーション風要約です。
全体として一貫したトピックになるように、日本語で5分程度（3,500〜4,000文字程度）でまとめ直してください。
語りかけるような自然な口調で、聞き手が全体像を把握できるように説明してください。

スライド要約一覧:
{text}

全体のナレーション（5分相当）:
""")

# --- チェーン実行（階層要約） ---
chain = load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=combine_prompt
)

narration_text = chain.run(docs)

print("✅ 生成されたナレーション:\n")
print(narration_text)

# --- OpenAI TTS で音声生成 ---
response = client.audio.speech.create(
    model="tts-1-hd",      # 高品質版（または tts-1）
    voice="shimmer",       # やわらかい女性声。alloy, nova, fable なども選択可
    input=narration_text
)

# --- 音声ファイル保存＆再生 ---
with open("narration_5min.mp3", "wb") as f:
    f.write(response.content)

# Mac/Linuxで再生（Windowsは "start narration_5min.mp3"）
os.system("afplay narration_5min.mp3")

✅ 生成されたナレーション:

こんにちは、皆さん。今日は、ビジネスの成長を後押しする要素であるデータを活用した販促戦略についてお話しします。具体的には、店頭接客の配置や動線、棚の配置、カゴの位置などを考慮し、VMDやアプリを活用して顧客との接触点を最大限に活用することが重要です。

ビジネスの成長を促進するためには、「集客」「トライアル」「リピート」「単価」「機会創出」の5つの要素が大切です。これらの施策を理解し、それぞれがどのように連携し、全体としてどのような効果をもたらすのかを把握することが必要です。また、ビジネスにおける目的変数の分類も重要です。これは、直接的なビジネス指標と間接的な指標に分けられ、それぞれが単一の指標や複数の指標の組み合わせで表されます。これらの指標を適切に把握し分析することで、ビジネスの現状理解や改善策の策定に役立てることができます。

店舗の売上を上げるためには、売上の要素を理解し、それぞれを向上させることが重要です。売上の要素とは、入館者数、前面通行量、認知率、認識率、入店率、滞在時間、買上げ率、一品単価などです。これら全てが組み合わさって、最終的な店舗の売上高や客単価が決まります。

さて、具体的な事例を見てみましょう。ドンキホーテでは、「呼び込み君」という録音機能を持つツールを活用しています。店舗担当者が特売商品などの販促コメントを録音し、店内で流すことで、来店客の耳に届き、注目すべき売り場に客の注意を引きます。また、＠cosme TOKYOでは、目立つ入口にベスコスタワーを設置し、店内にはスタッフがおすすめの商品や季節の商品を展示する「編集棚」を23カ所設けています。これらの展示は、＠cosme TOKYOのバイヤーや本社の商品部が毎月変更し、常に新鮮な商品ラインナップを提供しています。

さらに、生鮮食品と組み合わせた広告動画を活用することも効果的です。多くの買い物客は最初に生鮮食品の売り場に立ち寄り、そこでレシピを決め、その後に必要な調味料や飲料を買い足す傾向があります。そのため、生鮮食品売り場でレシピ動画を流すことで他のカテゴリの商品の価値も訴求できます。

最後に、九州のファミリーレストラン「ジョイフル」が、人流データ分析を手掛けるunerryと提携し、アプリを活用した新たな集客施策を展開した事例を紹介します。その結果、

sh: 1: afplay: not found


32512

In [21]:
import tiktoken
# トークン数カウント用
enc = tiktoken.encoding_for_model("gpt-4o")
tokens = enc.encode(narration_text)
token_count = len(tokens)

# 料金設定（2024年5月時点 GPT-4o）
gpt4o_input_rate = 0.0025  # $/1k tokens
gpt4o_output_rate = 0.01   # $/1k tokens
gpt4o_cost = (token_count / 1000) * (gpt4o_input_rate + gpt4o_output_rate)

# TTS 料金設定
tts_hd_rate = 0.03 / 1000  # $0.03/1k characters
tts_cost = len(narration_text) * tts_hd_rate

# 合計コスト
total_cost = gpt4o_cost + tts_cost

print(f"\n📊 見積もり（概算）:")
print(f"- GPT-4o トークン数: {token_count} tokens")
print(f"- GPT-4o 要約コスト: ${gpt4o_cost:.4f}")
print(f"- TTS 文字数: {len(narration_text)} chars")
print(f"- TTS 合成コスト: ${tts_cost:.4f}")
print(f"- 合計概算コスト: ${total_cost:.4f}")


📊 見積もり（概算）:
- GPT-4o トークン数: 1020 tokens
- GPT-4o 要約コスト: $0.0128
- TTS 文字数: 1360 chars
- TTS 合成コスト: $0.0408
- 合計概算コスト: $0.0536
